In [13]:
import pandas as pd
import numpy as np
import google.generativeai as genai
import os
from dotenv import load_dotenv

# load dot env
env_file_name = "C:/Users/LENOVO/Documents/GitHub/dtp-data-pipeline/gemini.env"

#fetch api gemini
if load_dotenv(dotenv_path=env_file_name):
    print(f"File{env_file_name} berhasil dimuat..")
else:
    print(f"File {env_file_name} tidak berhasil dimuat..")
    
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError(f"API KEY gagal dimuat pada file {env_file_name}..")
    
    
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-flash-lite')
print(f"konfig berhasil")

FileC:/Users/LENOVO/Documents/GitHub/dtp-data-pipeline/gemini.env berhasil dimuat..
konfig berhasil


In [14]:
try:
    df_lowongan = pd.read_csv("C:/Users/LENOVO/Documents/GitHub/dtp-data-pipeline/data_raw/Data Lowongan_1-3000.csv", sep=';')
    df_pon_tik = pd.read_csv("C:/Users/LENOVO/Documents/GitHub/dtp-data-pipeline/data_raw/PON TIK FIX - Sheet1.csv")
except FileNotFoundError as e:
    print(f"Error: {e}. Pastikan file CSV berada di direktori yang sama.")
    exit()

missing_values = ["Tidak Ada", "Tidak ada", "-", "", "Not Applicable"]

df_lowongan.replace(missing_values, np.nan, inplace=True)
print("Jumlah missing values awal di beberapa kolom:")
print(df_lowongan[['Level Pekerjaan', 'Industri', 'Spesial Info', 'Skillset', 'Tools']].isnull().sum())

Jumlah missing values awal di beberapa kolom:
Level Pekerjaan    786
Industri           189
Spesial Info         0
Skillset             8
Tools              308
dtype: int64


In [15]:
okupasi_to_level = df_pon_tik.set_index('OKUPASI')['LEVEL'].to_dict()

def impute_level_pekerjaan(row):
    if pd.notna(row['Level Pekerjaan']):
        return row['Level Pekerjaan']

    if pd.isna(row['Okupasi']):
        return None
        
    # clean okupasi row
    okupasi_lowongan = str(row['Okupasi'])
    
    okupasi_upper = okupasi_lowongan.upper()
    
    pos_separate = okupasi_upper.find(' - LEVEL')
    
    if pos_separate != -1:
        okupasi = okupasi_upper[:pos_separate].strip()
    else:
        okupasi = okupasi_upper.strip()

    for okupasi_pon_tik, level in okupasi_to_level.items():
        if okupasi in str(okupasi_pon_tik).upper():
            return level


    return None

df_lowongan['Level Pekerjaan'] = df_lowongan.apply(impute_level_pekerjaan, axis=1)

print("\nImputasi selesai")


Imputasi selesai


In [16]:
#pemetaan
def kategorisasi_level(level):
    try:
        level = int(float(level))
        if level <= 2:
            return 'Internship/Magang/OJT'
        elif level <= 4:
            return 'Lulusan Baru/Junior/Entry Level/Fresh Graduate'
        elif level == 5:
            return 'Associate'
        elif level == 6:
            return 'Mid Senior Level'
        elif level == 7:
            return 'Supervisor/Asisten Manager'
        elif level >= 8:
            return 'Direktur/Eksekutif'
        else:
            return np.nan
    except (ValueError, TypeError):
        return level

# categorized
df_lowongan['Level Pekerjaan'] = df_lowongan['Level Pekerjaan'].apply(kategorisasi_level)

print("Kategorisasi 'Level Pekerjaan' selesai.")
print("\nContoh hasil setelah perbaikan:")
print(df_lowongan[['Okupasi', 'Level Pekerjaan']].head())

Kategorisasi 'Level Pekerjaan' selesai.

Contoh hasil setelah perbaikan:
                         Okupasi  \
0        WEB DEVELOPER - Level 6   
1   IT SERVICE MANAGER - Level 7   
2         DATA ANALYST - Level 6   
3    SOFTWARE ENGINEER - Level 7   
4  ICT PROJECT MANAGER - Level 7   

                                  Level Pekerjaan  
0                    Supervisor / Asisten Manager  
1  Lulusan Baru/Junior/Entry Level/Fresh Graduate  
2                    Supervisor / Asisten Manager  
3  Lulusan Baru/Junior/Entry Level/Fresh Graduate  
4  Lulusan Baru/Junior/Entry Level/Fresh Graduate  


In [17]:
df_lowongan.head(100)

,No,Pekerjaan,Level Pekerjaan,Status Pekerjaan,Industri,Spesial Info,Deskripsi Pekerjaan,Jenis_Okupasi,Okupasi,Area Fungsi,Skillset,Tools
0,1,Developer (.Net/Java/ReactJs),Supervisor / Asisten Manager,FULL_TIME,IT and Software,IT and Software,"Write clean, secure, test driven, easily maint...",PON-TIK,WEB DEVELOPER - Level 6,Pengembangan Produk Digital,"Clean Code, Test Driven Development, Debugging...",".NET Framework, Java, ReactJS, unit testing fr..."
1,2,IT Vendor Management Staff,Lulusan Baru/Junior/Entry Level/Fresh Graduate,FULL_TIME,Insurance,IT and Software,Job description for IT Vendor Management will ...,PON-TIK,IT SERVICE MANAGER - Level 7,Layanan Teknologi Informasi,"Vendor Selection, Vendor Evaluation, Vendor Ne...","sistem manajemen vendor (misalnya SAP Ariba, O..."
2,3,Data Analyst (Product - Risk),Supervisor / Asisten Manager,FULL_TIME,Financial Services,IT and Software,"Research and analysis of the landscape, custom...",PON-TIK,DATA ANALYST - Level 6,Sains Data-Kecerdasan Artifisial,"Product Analysis, Trend Analysis, Business Str...","Spreadsheet Tools, Data Visualization Tools, P..."
3,4,Software Engineer,Lulusan Baru/Junior/Entry Level/Fresh Graduate,FULL_TIME,Information Technology / IT,IT and Software,Coding dan DebuggingMelakukan pengujian modul ...,PON-TIK,SOFTWARE ENGINEER - Level 7,Pengembangan Produk Digital,"Algorithm Design, Coding, Debugging, Problem ...","Database (MySQL, PostgreSQL), Server environme..."
4,5,IT Project Management Officer,Lulusan Baru/Junior/Entry Level/Fresh Graduate,FULL_TIME,Information Technology / IT,IT and Software,Oversee the development of the project and ens...,PON-TIK,ICT PROJECT MANAGER - Level 7,Tata Kelola Teknologi Informasi,"Project Oversight, Risk Management, Test Manag...","Project Management Tools, Risk Monitoring Tool..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,IT Project Staff,Lulusan Baru/Junior/Entry Level/Fresh Graduate,FULL_TIME,Consumer Goods,IT and Software,Identifying our business needsApplication supp...,PON-TIK,SYSTEM ENGINEER - Level 7,Teknologi Dan Infrastruktur,identifikasi kebutuhan;dukungan teknis;konfigu...,microsoft office;jira;servicenow;slack;trello
96,97,Network Engineer,Supervisor / Asisten Manager,FULL_TIME,Information Technology / IT,IT and Software,"Configure Router, Switch, Firewall and LoadBal...",PON-TIK,NETWORK ARCHITECT - Level 6,Teknologi Dan Infrastruktur,konfigurasi jaringan;keamanan jaringan;manajem...,router;switch;firewall;load balancer;ntp;syslo...
97,98,iOS Developer Tribunnews.com,Supervisor / Asisten Manager,FULL_TIME,Media Production,IT and Software,Design and build interactive and user-friendly...,PON-TIK,MOBILE PROGRAMMER - Level 7,Pengembangan Produk Digital,"iOS application development, Interactive app d...","iOS, Swift, Objective-C, Mobile applications"
98,99,Database Administrator,Supervisor / Asisten Manager,FULL_TIME,Information Technology / IT,IT and Software,Manage database changes and re-designsProvide ...,PON-TIK,DATABASE ADMINISTRATOR - Level 6,Sains Data-Kecerdasan Artifisial,"database redesign, problem analysis, incident ...",NaN


In [18]:
import time
import numpy as np
import pandas as pd

def impute_with_gemini_final(row, column_to_impute):
    if pd.notna(row[column_to_impute]):
        return row[column_to_impute]

    pekerjaan = row['Pekerjaan']
    deskripsi = row['Deskripsi Pekerjaan']

    if pd.isna(pekerjaan) or pd.isna(deskripsi):
        return np.nan
    
    # buat prompt
    prompts = {
        'Industri': f"Berdasarkan pekerjaan '{pekerjaan}' dan deskripsi '{deskripsi}', apa nama industri yang paling sesuai? Berikan satu jawaban singkat saja. Contoh: Teknologi Informasi.",
        'Spesial Info': f"Dari deskripsi pekerjaan '{deskripsi}' untuk posisi '{pekerjaan}', identifikasi special info yang memudahkan dalam mencari pekerjaan sesuai pada bidangn 'Contoh: Information Technology and Engineering'. Jika tidak ada, tulis 'Tidak ada'. Berikan jawaban singkat.",
        'Skillset': f"Berdasarkan deskripsi '{deskripsi}' untuk posisi '{pekerjaan}', sebutkan 5 skill utama yang dibutuhkan, pisahkan dengan titik koma. Contoh output: Web Development, Data Understanding, Tensorflow, PyTorch",
        'Tools': f"Berdasarkan deskripsi '{deskripsi}' untuk posisi '{pekerjaan}', sebutkan 3 tools/software utama yang digunakan, pisahkan dengan titik koma. Contoh output: Excel, Python, Power BI, MySQL"
    }
    
    prompt = prompts.get(column_to_impute)
    if not prompt:
        return np.nan

    try:
        response = model.generate_content(prompt)
        
        
        time.sleep(4) 
        
        return response.text.strip()
    except Exception as e:
        print(f"Error pada baris {row.name} untuk kolom '{column_to_impute}': {e}")
        return np.nan

In [19]:
columns_to_impute_gemini = ['Industri', 'Spesial Info', 'Skillset', 'Tools']

for col in columns_to_impute_gemini:
    print(f"\nMemulai imputasi penuh untuk kolom: {col} (ini akan memakan waktu)...")
    df_lowongan[col] = df_lowongan.apply(
        lambda row: impute_with_gemini_final(row, col) if pd.isna(row[col]) else row[col],
        axis=1
    )
    print(f"Imputasi untuk kolom '{col}' selesai.")

print("\nProses imputasi menggunakan Gemini telah selesai sepenuhnya.")


Memulai imputasi penuh untuk kolom: Industri (ini akan memakan waktu)...
Imputasi untuk kolom 'Industri' selesai.

Memulai imputasi penuh untuk kolom: Spesial Info (ini akan memakan waktu)...
Imputasi untuk kolom 'Spesial Info' selesai.

Memulai imputasi penuh untuk kolom: Skillset (ini akan memakan waktu)...
Imputasi untuk kolom 'Skillset' selesai.

Memulai imputasi penuh untuk kolom: Tools (ini akan memakan waktu)...
Imputasi untuk kolom 'Tools' selesai.

Proses imputasi menggunakan Gemini telah selesai sepenuhnya.


In [20]:
df_lowongan.isna().sum()

No                       0
Pekerjaan                0
Level Pekerjaan          8
Status Pekerjaan       106
Industri                 0
Spesial Info             0
Deskripsi Pekerjaan      0
Jenis_Okupasi            0
Okupasi                  0
Area Fungsi              0
Skillset                 0
Tools                    0
dtype: int64

In [21]:
import time
import numpy as np
import pandas as pd

def impute_with_gemini_final(row, column_to_impute):
    if pd.notna(row[column_to_impute]):
        return row[column_to_impute]

    pekerjaan = row['Pekerjaan']
    deskripsi = row['Deskripsi Pekerjaan']
    level = row['Level Pekerjaan']

    if pd.isna(pekerjaan) or pd.isna(deskripsi):
        return np.nan
    
    # buat prompt
    prompts = {
        'Level Pekerjaan': f"Berdasarkan pekerjaan '{pekerjaan}' dan deskripsi '{deskripsi}', apa level pekerjaan yang paling sesuai? Jawaban singkat terdiri dari Associate, Direktur/Eksekutif, Internship/Magang/OJT, Lulusan Baru/Junior/Entry Level/Fresh Graduate, Mid Senior Level, Supervisor/Asisten Manager.",
        'Status Pekerjaan': f"Dari deskripsi pekerjaan '{deskripsi}' untuk posisi '{pekerjaan}', dan level pekerjaan '{level}', apa status pekerjaan yang paling sesuai? Jawaban singkat terdiri dari Full Time, Kontrak/Temporer, Internship, Kasual, Part-Time.",
        'Tools': f"Berdasarkan deskripsi '{deskripsi}' untuk posisi '{pekerjaan}', sebutkan 3 tools/software utama yang digunakan, pisahkan dengan titik koma."
    }
    
    prompt = prompts.get(column_to_impute)
    if not prompt:
        return np.nan

    try:
        response = model.generate_content(prompt)
        
        
        time.sleep(4) 
        
        return response.text.strip()
    except Exception as e:
        print(f"Error pada baris {row.name} untuk kolom '{column_to_impute}': {e}")
        return np.nan

In [22]:
columns_to_impute_gemini = ['Level Pekerjaan', 'Status Pekerjaan']

for col in columns_to_impute_gemini:
    print(f"\nMemulai imputasi penuh untuk kolom: {col} (ini akan memakan waktu)...")
    df_lowongan[col] = df_lowongan.apply(
        lambda row: impute_with_gemini_final(row, col) if pd.isna(row[col]) else row[col],
        axis=1
    )
    print(f"Imputasi untuk kolom '{col}' selesai.")

print("\nProses imputasi menggunakan Gemini telah selesai sepenuhnya.")


Memulai imputasi penuh untuk kolom: Level Pekerjaan (ini akan memakan waktu)...
Imputasi untuk kolom 'Level Pekerjaan' selesai.

Memulai imputasi penuh untuk kolom: Status Pekerjaan (ini akan memakan waktu)...
Imputasi untuk kolom 'Status Pekerjaan' selesai.

Proses imputasi menggunakan Gemini telah selesai sepenuhnya.


In [24]:
df_lowongan.isna().sum()

No                     0
Pekerjaan              0
Level Pekerjaan        0
Status Pekerjaan       0
Industri               0
Spesial Info           0
Deskripsi Pekerjaan    0
Jenis_Okupasi          0
Okupasi                0
Area Fungsi            0
Skillset               0
Tools                  0
dtype: int64

In [26]:
df_lowongan.to_csv("C:\\Users\\LENOVO\\Documents\\GitHub\\dtp-data-pipeline\\data_processed\\1-3000\\Data_Lowongan_Pekerjaan_Cleaned_1-3000_Final.csv", index=False, encoding='utf-8-sig')